# Automated CapsnetsS2I train + test

First, import stuff

In [7]:
import model_s2i
import data_loader
import tensorflow as tf

In [8]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Load word vectors and data

In [37]:
WORD_VEC_SCENARIO = "FastText-300-verbsDatasets-synAntVSP-dia"
SCENARIO_NAME = 'o2-fasttext-100-mixed-1'

word2vec_path = '../../romanian_word_vecs/cleaned-vectors-diacritice-cc-100.vec'

# No diacritics data
training_data_paths = [
#         '../data-capsnets/scenario0/train.txt',
#         '../data-capsnets/scenario1/train.txt',
#         '../data-capsnets/scenario2/train.txt',
#         '../data-capsnets/scenario3.1/train.txt',
#         '../data-capsnets/scenario3.2/train.txt',
#         '../data-capsnets/scenario3.3/train.txt',
###############################################################
#         '../data-capsnets/diacritics/scenario0/train.txt',
#         '../data-capsnets/diacritics/scenario1/train.txt',
#         '../data-capsnets/diacritics/scenario2/train.txt',
#         '../data-capsnets/diacritics/scenario31/train.txt',
#         '../data-capsnets/diacritics/scenario32/train.txt',
#         '../data-capsnets/diacritics/scenario33/train.txt',
###############################################################
        '../data-capsnets/diacritics_mixed_1/scenario0/train.txt',
        '../data-capsnets/diacritics_mixed_1/scenario1/train.txt',
        '../data-capsnets/diacritics_mixed_1/scenario2/train.txt',
        '../data-capsnets/diacritics_mixed_1/scenario31/train.txt',
        '../data-capsnets/diacritics_mixed_1/scenario32/train.txt',
        '../data-capsnets/diacritics_mixed_1/scenario33/train.txt',
        
    ]
test_data_paths = [
#     '../data-capsnets/scenario0/test.txt',
#     '../data-capsnets/scenario1/test.txt',
#     '../data-capsnets/scenario2/test.txt',
#     '../data-capsnets/scenario3.1/test.txt',
#     '../data-capsnets/scenario3.2/test.txt',
#     '../data-capsnets/scenario3.3/test.txt',
###############################################################
#     '../data-capsnets/diacritics/scenario0/test.txt',
#     '../data-capsnets/diacritics/scenario1/test.txt',
#     '../data-capsnets/diacritics/scenario2/test.txt',
#     '../data-capsnets/diacritics/scenario31/test.txt',
#     '../data-capsnets/diacritics/scenario32/test.txt',
#     '../data-capsnets/diacritics/scenario33/test.txt',
###############################################################
    '../data-capsnets/diacritics_mixed_1/scenario0/test.txt',
    '../data-capsnets/diacritics_mixed_1/scenario1/test.txt',
    '../data-capsnets/diacritics_mixed_1/scenario2/test.txt',
    '../data-capsnets/diacritics_mixed_1/scenario31/test.txt',
    '../data-capsnets/diacritics_mixed_1/scenario32/test.txt',
    '../data-capsnets/diacritics_mixed_1/scenario33/test.txt',
]

scenario_nums = [
#     '0_' + WORD_VEC_SCENARIO,
#     '1_' + WORD_VEC_SCENARIO,
#     '2_' + WORD_VEC_SCENARIO,
#     '31_' + WORD_VEC_SCENARIO,
#     '32_' + WORD_VEC_SCENARIO,
#     '33_' + WORD_VEC_SCENARIO,
#     '0_dia_' + WORD_VEC_SCENARIO,
#     '1_dia_' + WORD_VEC_SCENARIO,
#     '2_dia_' + WORD_VEC_SCENARIO,
#     '31_dia_' + WORD_VEC_SCENARIO,
#     '32_dia_' + WORD_VEC_SCENARIO,
#     '33_dia_' + WORD_VEC_SCENARIO,
    '0-' + SCENARIO_NAME,
    '1-' + SCENARIO_NAME,
    '2-' + SCENARIO_NAME,
    '31-' + SCENARIO_NAME,
    '32-' + SCENARIO_NAME,
    '33-' + SCENARIO_NAME,
]

In [38]:
print('------------------load word2vec begin-------------------')
w2v = data_loader.load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
loading time took 000.04
------------------load word2vec end---------------------


# Flag setting functions + utils

In [39]:
import csv

def set_flags(data, scenario):
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
    tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
    tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
    tf.app.flags.DEFINE_string('scenario_num', scenario, 'Scenario number')
    tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
    tf.app.flags.DEFINE_string('results_dir', './results/', 'Results dir')
    tf.app.flags.DEFINE_string('hyperparams_dir', './hyperparams/', 'Hyperparams dir')
    tf.app.flags.DEFINE_string('confidence_dir', './conf_levels/', 'Confidence levels analysis dir')
    
    vocab_size, word_emb_size = data['embedding'].shape
    _, max_sentence_length = data['x_tr'].shape
    intents_number = len(data['intents_dict'])
    slots_number = len(data['slots_dict'])
    hidden_size = 16

    tf.app.flags.DEFINE_float('keep_prob', 1, 'embedding dropout keep rate for training')
    tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
    tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
    tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
    tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
    tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
    tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
    tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
    tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')
    tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
    tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
    tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
    tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
    tf.app.flags.DEFINE_integer('intent_output_dim', 8, 'intent output dimension')
    tf.app.flags.DEFINE_integer('slot_output_dim', 2 * hidden_size, 'slot output dimension')
    tf.app.flags.DEFINE_integer('d_a', 10, 'self attention weight hidden units number')
    tf.app.flags.DEFINE_integer('r', 3, 'number of self attention heads')
    tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
    tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
    tf.app.flags.DEFINE_float('rerouting_coef', 0.5, 'coefficient for rerouting')
    tf.app.flags.DEFINE_boolean('use_rerouting', True, 'whether to use rerouting or not')
    tf.app.flags.DEFINE_boolean('use_attention', True,
                                'whether to use attention or not. If attention is used,'
                                'slot_output_dim will be overridden to hidden_size * 2')
    
    tf.app.flags.DEFINE_string('f', '', 'kernel') # Without this there's an error: unknown command line flag 'f'
    
    return FLAGS

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
def write_csv(run_results):
    with open('results/results_{}.csv'.format(SCENARIO_NAME), 'w') as f:
        header_line = ['Scenario', 'Intent F1', 'Slot F1']
        writer = csv.writer(f)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            intent_score = [v['intent_f1']]
            slot_score = [v['slot_f1']]
            l = scenario_num + intent_score + slot_score
            writer.writerow(l)

In [32]:
# In case it's needed
del_all_flags(FLAGS)

In [40]:
import train
import test

results = dict()
for i in range(len(training_data_paths)):
    
    # Load data
    isLowercase = False # When using the new 100-dim word vec model (conll, not fasttext), the data should all be in lowercase
    data = data_loader.read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True, lowercase=isLowercase)
    
    FLAGS = set_flags(data, scenario=scenario_nums[i])
    
    # Train model
    print('TRAINING ' + scenario_nums[i])
    train.train(model_s2i.CapsNetS2I, data, FLAGS, batches_rand=False)
    
    # Test model
    print('TESTING ' + scenario_nums[i])
    intent_f, slot_f = test.test(model_s2i.CapsNetS2I, data, FLAGS)
    results[scenario_nums[i]] = dict()
    results[scenario_nums[i]]['intent_f1'] = intent_f
    results[scenario_nums[i]]['slot_f1'] = slot_f
        
    # Reset flags
    del_all_flags(FLAGS)

write_csv(results)

------------------read datasets begin-------------------
max length is 15
max length is 15
------------------read datasets end---------------------
TRAINING 0-o2-fasttext-100-mixed-1
------------------start cross-validation-------------------
FOLD 1
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.071429
F score 0.071429
Slot filling
F1 score: 0.009276
Accuracy: 0.024286
------------------epoch :  0  Loss:  26.60309 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.314286
F score 0.314286
Slot filling
F1 score: 0.083077
Accuracy: 0.899206
Current F score mean 0.19868131868131866
Best F score mean 0.19868131868131866
------------------epoch :  1  Loss:  9.221556 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.721429
F score 0.721429
Slot fi

------------------epoch :  5  Loss:  0.32133412 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.878571
F score 0.878571
Slot filling
F1 score: 0.900200
Accuracy: 0.992540
Current F score mean 0.8893855146849159
Best F score mean 0.9895054573553559
------------------epoch :  6  Loss:  0.1429672 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.902381
F score 0.902381
Slot filling
F1 score: 0.912000
Accuracy: 0.993810
Current F score mean 0.9071904761904761
Best F score mean 0.9895054573553559
------------------epoch :  7  Loss:  0.097744025 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.933333
F score 0.933333
Slot filling
F1 score: 0.950355
Accuracy: 0.995397
Current F score mean 0.9418439716312057
Best F score mean 0.9895054573553559
------------------epoch :  8  Loss:  0.09458648 ----------------------
           VALIDA

------------------epoch :  11  Loss:  0.014457441 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.997619
F score 0.997619
Slot filling
F1 score: 0.990844
Accuracy: 0.998889
Current F score mean 0.9942317008186794
Best F score mean 0.9942317008186794
------------------epoch :  12  Loss:  0.012840223 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.995238
F score 0.995238
Slot filling
F1 score: 0.990844
Accuracy: 0.998889
Current F score mean 0.9930412246282032
Best F score mean 0.9942317008186794
------------------epoch :  13  Loss:  0.027207555 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.997619
F score 0.997619
Slot filling
F1 score: 0.992879
Accuracy: 0.999048
Current F score mean 0.9952489948166449
Best F score mean 0.9952489948166449
------------------epoch :  14  Loss:  0.039748378 ----------------------
         

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.989899
Accuracy: 0.998980
Current F score mean 0.9889971139971141
Best F score mean 0.9889971139971141
------------------epoch :  12  Loss:  0.008251678 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.986882
Accuracy: 0.998810
Current F score mean 0.9874885877660853
Best F score mean 0.9889971139971141
------------------epoch :  13  Loss:  0.0072383415 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.986882
Accuracy: 0.998810
Current F score mean 0.9874885877660853
Best F score mean 0.9889971139971141
------------------epoch :  14  Loss:  0.0064106225 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.974824
Accuracy: 0.997449
Current F score mean 0.9814595022298951
Best F score mean 0.9889971139971141
------------------epoch :  18  Loss:  0.004712221 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.974824
Accuracy: 0.997449
Current F score mean 0.9814595022298951
Best F score mean 0.9889971139971141
------------------epoch :  19  Loss:  0.00435073 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.974824
Accuracy: 0.997449
Current F score mean 0.9814595022298951
Best F score mean 0.9889971139971141
FOLD 3
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE   

...

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.914454
F score 0.914454
Slot filling
F1 score: 0.951220
Accuracy: 0.993890
Current F score mean 0.9328368947406288
Best F score mean 0.9805460781177808
------------------epoch :  5  Loss:  0.12652543 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.946903
F score 0.946903
Slot filling
F1 score: 0.948012
Accuracy: 0.994522
Current F score mean 0.9474574436415795
Best F score mean 0.9805460781177808
------------------epoch :  6  Loss:  0.08053513 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.976401
F score 0.976401
Slot filling
F1 score: 0.967841
Accuracy: 0.995786
Current F score mean 0.9721209575049579
Best F score mean 0.9805460781177808
------------------epoch :  7  Loss:  0.034412663 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.991150

------------------epoch :  11  Loss:  0.012065615 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.985251
F score 0.985251
Slot filling
F1 score: 0.936170
Accuracy: 0.994311
Current F score mean 0.9607104751145421
Best F score mean 0.984871857324565
------------------epoch :  12  Loss:  0.010663662 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.985251
F score 0.985251
Slot filling
F1 score: 0.943511
Accuracy: 0.994522
Current F score mean 0.9643810939224031
Best F score mean 0.984871857324565
------------------epoch :  13  Loss:  0.009573764 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.985251
F score 0.985251
Slot filling
F1 score: 0.942073
Accuracy: 0.994522
Current F score mean 0.9636619540974171
Best F score mean 0.984871857324565
------------------epoch :  14  Loss:  0.008438833 ----------------------
           V

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.961039
F score 0.961039
Slot filling
F1 score: 0.967922
Accuracy: 0.995053
Current F score mean 0.9644804289155753
Best F score mean 0.9644804289155753
------------------epoch :  12  Loss:  0.08144233 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.965368
F score 0.965368
Slot filling
F1 score: 0.969274
Accuracy: 0.995362
Current F score mean 0.9673208541923626
Best F score mean 0.9673208541923626
------------------epoch :  13  Loss:  0.061376877 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.961039
F score 0.961039
Slot filling
F1 score: 0.972067
Accuracy: 0.995671
Current F score mean 0.9665530000725531
Best F score mean 0.9673208541923626
------------------epoch :  14  Loss:  0.04647002 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.965

------------------epoch :  18  Loss:  0.019330455 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.974026
F score 0.974026
Slot filling
F1 score: 0.975069
Accuracy: 0.995671
Current F score mean 0.9745476130517682
Best F score mean 0.9752238053208927
------------------epoch :  19  Loss:  0.01739202 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.974026
F score 0.974026
Slot filling
F1 score: 0.975069
Accuracy: 0.995671
Current F score mean 0.9745476130517682
Best F score mean 0.9752238053208927
FOLD 3
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.086580
F score 0.086580
Slot filling
F1 score: 0.022862
Accuracy: 0.033704
------------------epoch :  0  Loss:  32.23813 ----------------------
           VALIDATION SET PERFORMANCE        
In

Intent F1: 0.533095
Slot F1: 0.485513
